# Multi-factor ANOVAs to support PPI analyses

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_ind, ttest_1samp

root = '/Users/haileytrier/Downloads/Trier_et_al_2023_code/data/'

def getPeakVals(df, reg_name, search_window, ROI, seed):
    '''
    Extract peak values from the spreadsheet for a particular regressor within a search window.
    df: data from csv file.
    reg_name: name of regressor desired.
    '''
    cols = [x for x in df.columns if 'Peak_vals' in x]
    peaks = df.loc[(df.Regressor==reg_name)&(df.Peak_search_window==search_window)&(df.ROI==ROI)&(df.Seed==seed),cols].values[0].copy()
    pval = df.loc[(df.Regressor==reg_name)&(df.Peak_search_window==search_window)&(df.ROI==ROI)&(df.Seed==seed),'Uncorrected_pvalue'].values[0].copy()
    sig = True if pval < 0.05 else False
    return peaks, sig

def getPeakVals_noSeed(df, reg_name, search_window, ROI):
    '''
    Extract peak values from the spreadsheet for a particular regressor within a search window.
    df: data from csv file.
    reg_name: name of regressor desired.
    '''
    cols = [x for x in df.columns if 'Peak_vals' in x]
    peaks = df.loc[(df.Regressor==reg_name)&(df.Peak_search_window==search_window)&(df.ROI==ROI),cols].values[0].copy()
    pval = df.loc[(df.Regressor==reg_name)&(df.Peak_search_window==search_window)&(df.ROI==ROI),'Uncorrected_pvalue'].values[0].copy()
    sig = True if pval < 0.05 else False
    return peaks, sig

def addFactorsToDataframe(df, seed:str=np.nan, ROI:str=np.nan, peaks:float=np.nan, 
                          win:str=np.nan, success:str=np.nan, phase:str=np.nan, action:str=np.nan):
    '''
    Add ROI and seed as factors in the dataframe along with peak values.
    '''
    df = pd.concat([df,pd.DataFrame(data={
        'Seed':[seed for x in range(len(peaks))],
        'ROI':[ROI for x in range(len(peaks))],
        'Peaks':peaks,
        'Search_window':[win for x in range(len(peaks))],
        'Success':success,
        'Phase':phase,
        'Action':action
    })])
    df = df.reset_index(drop=True)
    return df.copy()

### ANOVA comparing ROI, action type, and regressor

In [ ]:
allcopes = pd.read_csv('/Users/haileytrier/Downloads/Trier_et_al_2023_code/data/TableS6.csv')

df = allcopes[allcopes['ROI'].isin(['DRN','HB','VTA','SN']) & # 'ACC','AI'
              allcopes['Action'].isin(['Switch to check', 'Switch to forage']) & 
              allcopes['Phase'].isin(['pre-PD']) &
              allcopes['Regressor'].isin(['Reward','TimePressure'])]
df = df.reset_index(drop=True)
df

In [ ]:
model = ols('Value ~ C(ROI) + C(Regressor) + C(Action) + C(ROI)*C(Regressor)*C(Action)', data=df).fit()
sm.stats.anova_lm(model, typ=2)

### 2-way ANOVA testing the effects of cortical region (ACC/AI) and sub-cortical region (Hb/DRN) on the extent to which check switches moderated functional connectivity

In [ ]:
# Load PPI results from analyses that produced Fig. 4A
FC_threat_2sPre = pd.read_csv(root+'PPIs_switchToCheck_Threat_2sPre.csv')
FC_threat_5sPre = pd.read_csv(root+'PPIs_switchToCheck_Threat_5sPre.csv')

In [ ]:
earlywin = '0.06s-4.98s'
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])

# Effect of ACC*Check on HB
peaks, sig = getPeakVals(FC_threat_5sPre,'PPI1_seedTCXswitchToCheck','-5s-5s',ROI='HB',seed='ACC_cluster_cope7')
df = addFactorsToDataframe(df, seed='ACC', ROI='HB',peaks=peaks,win='-5s-5s',phase='early')
peaks, sig = getPeakVals(FC_threat_5sPre,'PPI1_seedTCXswitchToCheck','5s-10s',ROI='HB',seed='ACC_cluster_cope7')
df = addFactorsToDataframe(df, seed='ACC', ROI='HB',peaks=peaks,win='5s-10s',phase='late')

# Effect of AI*Check on HB
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',earlywin,ROI='HB',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI', ROI='HB',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',latewin,ROI='HB',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI', ROI='HB',peaks=peaks,win=latewin,phase='late')

# Effect of ACC*Check on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',earlywin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="ACC_cluster_cope7")
df = addFactorsToDataframe(df, seed='ACC',ROI='DRN',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',latewin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="ACC_cluster_cope7")
df = addFactorsToDataframe(df, seed='ACC',ROI='DRN',peaks=peaks,win=latewin,phase='late')

# Effect of AI*Check on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',earlywin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI',ROI='DRN',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',latewin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI',ROI='DRN',peaks=peaks,win=latewin,phase='late')

df

In [ ]:
# Table S9
print("NOTE: DATA FROM EARLY AND LATE WINDOWS ENTERED")
model = ols('Peaks ~ C(Seed) + C(ROI) + C(Seed)*C(ROI)', data=df).fit()
sm.stats.anova_lm(model, typ=2)

In [ ]:
# Table S8
# Post-hoc comparisons to examine the main effect of Seed
print(df.loc[:,['Seed','Peaks']].groupby('Seed').describe())

print("\nTukey test:")
tukey = pairwise_tukeyhsd(endog=df['Peaks'],
                          groups=df['Seed'],
                          alpha=0.05)
print(tukey)

In [ ]:
# Main Text
# Descriptive stats for identifying panel that is different according to significant interaction
print(df.loc[:,['Seed','ROI','Peaks']].groupby(['Seed','ROI']).describe())

### 2-way ANOVA testing the effects of cortical (ACC/AI) and sub-cortical (DRN/Hb) on the extent to which check switches and time pressure moderated functional connectivity.

In [ ]:
earlywin = '0.06s-4.98s'
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])

# Effect of ACC*Check*Threat on HB
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI='HB',seed='ACC_cluster_cope7')
df = addFactorsToDataframe(df, seed='ACC', ROI='HB',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI='HB',seed='ACC_cluster_cope7')
df = addFactorsToDataframe(df, seed='ACC', ROI='HB',peaks=peaks,win=latewin,phase='late')

# Effect of AI*Check*Threat on HB
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI='HB',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI', ROI='HB',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI='HB',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI', ROI='HB',peaks=peaks,win=latewin,phase='late')

# Effect of ACC*Check*Threat on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="ACC_cluster_cope7")
df = addFactorsToDataframe(df, seed='ACC',ROI='DRN',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="ACC_cluster_cope7")
df = addFactorsToDataframe(df, seed='ACC',ROI='DRN',peaks=peaks,win=latewin,phase='late')

# Effect of AI*Check*Threat on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI',ROI='DRN',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI',ROI='DRN',peaks=peaks,win=latewin,phase='late')

df

In [ ]:
# Table S10
print("NOTE: PEAK DATA FROM WHOLE WINDOW ENTERED")
model = ols('Peaks ~ C(Seed) + C(ROI) + C(Seed)*C(ROI)', data=df).fit()
sm.stats.anova_lm(model, typ=2) # Type 2 Anova DataFrame

In [ ]:
# Table S11
# Post-hoc comparisons to examine the main effect of sub-cortical ROI
print(df.loc[:,['ROI','Peaks']].groupby('ROI').describe())

print("\nTukey test:")
tukey = pairwise_tukeyhsd(endog=df['Peaks'],
                          groups=df['ROI'],
                          alpha=0.05)
print(tukey)

In [ ]:
# Table S12
# Post-hoc comparisons to examine the main effect of cortical ROI
print(df.loc[:,['Seed','Peaks']].groupby('Seed').describe())

print("\nTukey test:")
tukey = pairwise_tukeyhsd(endog=df['Peaks'],
                          groups=df['Seed'],
                          alpha=0.05)
print(tukey)

### Two-way ANOVA testing the effects of ROI and behavioral switch on the extent to which time pressure moderated functional connectivity with Hb.

In [ ]:
forageSwitch_threat = pd.read_csv(root+'PPIs_switchToForage_Threat_2sPre.csv')

In [ ]:
# Read in PPI results related to switch to checking

earlywin = '0.06s-4.98s'
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])
action = "check" # reward

# Effect of HB*Threat*Check on SN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',seed="HB",peaks=peaks,win=earlywin,action=action,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',seed="HB",peaks=peaks,win=latewin,action=action,phase='late')

# Effect of HB*Threat*Check on VTA
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',seed="HB",peaks=peaks,win=earlywin,action=action,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',seed="HB",peaks=peaks,win=latewin,action=action,phase='late')

# Effect of HB*Threat*Check on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',seed="HB",peaks=peaks,win=earlywin,action=action,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',seed="HB",peaks=peaks,win=latewin,action=action,phase='late')

df

In [ ]:
# Read in PPI results related to switch to foraging

# Effect of HB*Reward*Forage on SN
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',earlywin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',seed="HB",peaks=peaks,win=earlywin,action='forage',phase='early')
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',latewin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',seed="HB",peaks=peaks,win=latewin,action='forage',phase='late')

# Effect of HB*Reward*Forage on VTA
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',earlywin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',seed="HB",peaks=peaks,win=earlywin,action='forage',phase='early')
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',latewin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',seed="HB",peaks=peaks,win=latewin,action='forage',phase='late')

# Effect of HB*Reward*Forage on DRN
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',seed="HB",peaks=peaks,win=earlywin,action='forage',phase='early')
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',seed="HB",peaks=peaks,win=latewin,action='forage',phase='late')

df

In [ ]:
# Table S13
model = ols('Peaks ~ C(ROI) + C(Action) + C(ROI)*C(Action)', data=df).fit()
sm.stats.anova_lm(model, typ=2)

### 2-way ANOVA testing the effects of ROI and check success on the extent to which time pressure moderated functional connectivity.

In [2]:
successFC_threat = pd.read_csv(root+'PPIs_successfulSwitchToCheck_Threat_2sPre.csv')
failFC_threat = pd.read_csv(root+'PPIs_unsuccessfulSwitchToCheck_Threat_2sPre.csv')

In [3]:
earlywin = '0.06s-4.98s'
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])

# Effect of HB*Threat*Check (success) on SN
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',peaks=peaks,win=earlywin,success='Success',phase='early')
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',peaks=peaks,win=latewin,success='Success',phase='late')

# Effect of HB*Threat*Check (success) on VTA
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',peaks=peaks,win=earlywin,success='Success',phase='early')
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',peaks=peaks,win=latewin,success='Success',phase='late')

# Effect of HB*Threat*Check (success) on DRN
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',peaks=peaks,win=earlywin,success='Success',phase='early')
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',peaks=peaks,win=latewin,success='Success',phase='late')

# Effect of HB*Threat*Check (fail) on SN
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',peaks=peaks,win=earlywin,success='Fail',phase='early')
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',peaks=peaks,win=latewin,success='Fail',phase='late')

# Effect of HB*Threat*Check (fail) on VTA
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',peaks=peaks,win=earlywin,success='Fail',phase='early')
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',peaks=peaks,win=latewin,success='Fail',phase='late')

# Effect of HB*Threat*Check (fail) on DRN
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',peaks=peaks,win=earlywin,success='Fail',phase='early')
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',peaks=peaks,win=latewin,success='Fail',phase='late')

df

/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_38083/4130225103.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame(data={


,Seed,ROI,Peaks,Search_window,Success,Phase,Action
0,NaN,SN,-0.224,0.06s-4.98s,Success,early,NaN
1,NaN,SN,0.031,0.06s-4.98s,Success,early,NaN
2,NaN,SN,0.134,0.06s-4.98s,Success,early,NaN
3,NaN,SN,-0.134,0.06s-4.98s,Success,early,NaN
4,NaN,SN,0.011,0.06s-4.98s,Success,early,NaN
...,...,...,...,...,...,...,...
271,NaN,DRN,0.007,4.98s-9.99s,Fail,late,NaN
272,NaN,DRN,-0.017,4.98s-9.99s,Fail,late,NaN
273,NaN,DRN,-0.087,4.98s-9.99s,Fail,late,NaN
274,NaN,DRN,0.032,4.98s-9.99s,Fail,late,NaN


In [4]:
# Table S15
print("NOTE: PEAK DATA FROM WHOLE WINDOW ENTERED")
model = ols('Peaks ~ C(ROI) + C(Success) + C(ROI)*C(Success)', data=df.loc[df.Phase=='late']).fit()
sm.stats.anova_lm(model, typ=2) # Type 2 Anova DataFrame

NOTE: PEAK DATA FROM WHOLE WINDOW ENTERED


,sum_sq,df,F,PR(>F)
C(ROI),0.007,2.000,0.194,0.824
C(Success),0.406,1.000,23.775,0.000
C(ROI):C(Success),0.008,2.000,0.222,0.801
Residual,2.252,132.000,NaN,NaN


### Confirm whether peak values for HbxCheckxTimepressure and DRNxCheckxTimepressure during successful checks are more negative than those for VTAxCheckxTimepressure

In [ ]:
successFC_threat = pd.read_csv(root+'PPIs_successfulSwitchToCheck_Threat_2sPre.csv')
peak_times = pd.read_csv(root+'/PPI_peak_times_subcortical_check-timepressure-HB.csv')

In [ ]:
# Analyze the number of unique peak times per ROI
peak_cols = [x for x in peak_times.columns if 'Peak_time_' in x]

vta_times = np.unique(peak_times.loc[peak_times['ROI']=='VTA',peak_cols].values)
print("Unique times for VTA peaks:\n",vta_times)
print(f'Mean: {np.mean(vta_times)}, SD={np.std(vta_times)}')

sn_times = np.unique(peak_times.loc[peak_times['ROI']=='SN',peak_cols].values)
print("\n\nUnique times for SN peaks:\n",sn_times)
print(f'Mean: {np.mean(sn_times)}, SD={np.std(sn_times)}')

drn_times = np.unique(peak_times.loc[peak_times['ROI']=='DRN',peak_cols].values)
print("\n\nUnique times for DRN peaks:\n",drn_times)
print(f'Mean: {np.mean(drn_times)}, SD={np.std(drn_times)}')


In [ ]:
min(vta_times)#-min(vta_times)

In [ ]:
# Format peak times for anova
peak_times_stacked = peak_times.loc[:,peak_cols].T.reset_index().rename(columns={0:'SN',1:'VTA',2:'DRN'})
peak_times_stacked = peak_times_stacked.loc[:,['SN','VTA','DRN']].stack()
peak_times_stacked = peak_times_stacked.reset_index(level=0, drop=True).rename_axis(index={None: 'ROI'}).reset_index()
peak_times_stacked = peak_times_stacked.rename(columns={0: "Times"})
peak_times_stacked

In [ ]:
model = ols('Times ~ C(ROI)', data=peak_times_stacked).fit()
sm.stats.anova_lm(model, typ=2) # Type 2 Anova DataFrame

In [ ]:
# Post-hoc comparisons to examine the main effect of ROI
print(peak_times_stacked.groupby('ROI').describe())

print("\nTukey test:")
tukey = pairwise_tukeyhsd(endog=peak_times_stacked['Times'],
                          groups=peak_times_stacked['ROI'],
                          alpha=0.05)
print(tukey)

In [ ]:
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])

# Effect of HB*Threat*Check (success) on SN
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',peaks=peaks,win=latewin,success='Success',phase='late',seed='HB')

# Effect of HB*Threat*Check (success) on VTA
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',peaks=peaks,win=latewin,success='Success',phase='late',seed='HB')

# Effect of HB*Threat*Check (success) on DRN
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',peaks=peaks,win=latewin,success='Success',phase='late',seed='HB')


In [ ]:
print("NOTE: PEAK DATA FROM ONLY LATE WINDOW ENTERED")
model = ols('Peaks ~ C(ROI)', data=df).fit()
sm.stats.anova_lm(model, typ=2) # Type 2 Anova DataFrame

### Compare Hb & DRN values for check constant in predator discovery vs. non-discovery conditions

In [2]:
successfulCheckConstant_prePD = pd.read_csv(f'{root}PPIs_successfulPredatorDiscovery_prePD_CheckOnly_DRN_HB.csv')
unsuccessfulCheckConstant_prePD = pd.read_csv(f'{root}PPIs_unsuccessfulPredatorDiscovery_prePD_CheckOnly_DRN_HB.csv')
unsuccessfulCheckConstant_prePD.head()

,Condition,Regressor,ROI,Seed,Uncorrected_pvalue,Significant,T_stats,df,Avg_peak_sec,Peak_search_window,...,Peak_vals_14,Peak_vals_15,Peak_vals_16,Peak_vals_17,Peak_vals_18,Peak_vals_19,Peak_vals_20,Peak_vals_21,Peak_vals_22,Peak_vals_23
0,vigilance_unsuccessfulChecks,timepressure,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.000,1,7.302,22,2.717,0.06s-4.98s,...,0.145,0.075,0.028,-0.053,0.221,0.075,0.046,0.176,0.126,0.227
1,vigilance_unsuccessfulChecks,time,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.453,0,-0.764,22,3.994,0.06s-4.98s,...,-0.013,-0.047,0.093,0.017,-0.196,-0.011,-0.196,-0.049,0.194,-0.057
2,vigilance_unsuccessfulChecks,reward,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.506,0,-0.676,22,2.618,0.06s-4.98s,...,0.053,-0.117,-0.046,-0.033,0.153,-0.040,0.051,0.034,-0.015,0.174
3,vigilance_unsuccessfulChecks,constant,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.009,1,2.844,22,2.815,0.06s-4.98s,...,0.035,-0.283,0.298,0.213,0.147,0.167,0.147,0.020,0.221,0.096
4,vigilance_unsuccessfulChecks,timepressure,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.001,1,-4.018,22,9.301,4.98s-9.99s,...,-0.041,-0.134,-0.069,-0.025,-0.093,-0.077,-0.115,-0.081,-0.205,0.048


In [3]:
# Read in PPI results related to switch to checking
earlywin = '0.06s-4.98s'
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])
action = "check"

# Effect of successful check constant, HB
peaks, sig = getPeakVals(successfulCheckConstant_prePD,'constant',earlywin,ROI="HB",seed='none')
df = addFactorsToDataframe(df,ROI='Hb',seed='none',peaks=peaks,win=earlywin,action=action,phase='early',success=True)
peaks, sig = getPeakVals(successfulCheckConstant_prePD,'constant',latewin,ROI="HB",seed='none')
df = addFactorsToDataframe(df,ROI='Hb',seed='none',peaks=peaks,win=latewin,action=action,phase='late',success=True)

# Effect of unsuccessful check constant, HB
peaks, sig = getPeakVals(unsuccessfulCheckConstant_prePD,'constant',earlywin,ROI="HB",seed='none')
df = addFactorsToDataframe(df,ROI='Hb',seed='none',peaks=peaks,win=earlywin,action=action,phase='early',success=False)
peaks, sig = getPeakVals(unsuccessfulCheckConstant_prePD,'constant',latewin,ROI="HB",seed='none')
df = addFactorsToDataframe(df,ROI='Hb',seed='none',peaks=peaks,win=latewin,action=action,phase='late',success=False)

# Effect of successful check constant, DRN
peaks, sig = getPeakVals(successfulCheckConstant_prePD,'constant',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed='none')
df = addFactorsToDataframe(df,ROI='DRN',seed='none',peaks=peaks,win=earlywin,action=action,phase='early',success=True)
peaks, sig = getPeakVals(successfulCheckConstant_prePD,'constant',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed='none')
df = addFactorsToDataframe(df,ROI='DRN',seed='none',peaks=peaks,win=latewin,action=action,phase='late',success=True)

# Effect of unsuccessful check constant, HB
peaks, sig = getPeakVals(unsuccessfulCheckConstant_prePD,'constant',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed='none')
df = addFactorsToDataframe(df,ROI='DRN',seed='none',peaks=peaks,win=earlywin,action=action,phase='early',success=False)
peaks, sig = getPeakVals(unsuccessfulCheckConstant_prePD,'constant',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed='none')
df = addFactorsToDataframe(df,ROI='DRN',seed='none',peaks=peaks,win=latewin,action=action,phase='late',success=False)

df

/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_53383/2743235282.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame(data={


,Seed,ROI,Peaks,Search_window,Success,Phase,Action
0,none,Hb,0.406,0.06s-4.98s,True,early,check
1,none,Hb,0.296,0.06s-4.98s,True,early,check
2,none,Hb,0.326,0.06s-4.98s,True,early,check
3,none,Hb,0.531,0.06s-4.98s,True,early,check
4,none,Hb,0.276,0.06s-4.98s,True,early,check
...,...,...,...,...,...,...,...
179,none,DRN,-0.031,4.98s-9.99s,False,late,check
180,none,DRN,-0.147,4.98s-9.99s,False,late,check
181,none,DRN,-0.007,4.98s-9.99s,False,late,check
182,none,DRN,-0.046,4.98s-9.99s,False,late,check


In [4]:
# Test effect of ROI (HB, DRN) and check success (True, False) on peak values for the check constant
model = ols('Peaks ~ C(ROI) + C(Success) + C(ROI)*C(Success)', data=df).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(ROI),1.431,1.000,60.942,0.000
C(Success),0.043,1.000,1.817,0.179
C(ROI):C(Success),2.868,1.000,122.111,0.000
Residual,4.228,180.000,NaN,NaN


In [5]:
# Descriptive stats for discovering significant interaction between ROI and check success
print(df.loc[:,['Success','ROI','Peaks']].groupby(['Success','ROI']).describe())

             Peaks                                               
             count   mean   std    min    25%    50%    75%   max
Success ROI                                                      
False   DRN 46.000  0.051 0.108 -0.283 -0.026  0.047  0.132 0.298
        Hb  46.000 -0.022 0.131 -0.441 -0.106 -0.019  0.058 0.273
True    DRN 46.000 -0.168 0.199 -0.426 -0.306 -0.189 -0.071 0.409
        Hb  46.000  0.258 0.160 -0.036  0.148  0.290  0.354 0.617


In [6]:
# Independent t-test: pre-PD Hb predator discovery vs. no-discovery peak values
ttest_ind(
    a = df.loc[
        (df.ROI=='Hb') &
        (df.Success==True),'Peaks'
    ].values,
    b = df.loc[
        (df.ROI=='Hb') &
        (df.Success==False),'Peaks'
    ].values,
nan_policy='omit')

TtestResult(statistic=9.185709593536199, pvalue=1.4132341996619983e-14, df=90.0)

In [7]:
# Independent t-test: pre-PD DRN predator discovery vs. no-discovery peak values
ttest_ind(
    a = df.loc[
        (df.ROI=='DRN') &
        (df.Success==True),'Peaks'
    ].values,
    b = df.loc[
        (df.ROI=='DRN') &
        (df.Success==False),'Peaks'
    ].values,
nan_policy='omit')

TtestResult(statistic=-6.574074456691293, pvalue=3.120589441428147e-09, df=90.0)

### Test pre-PD versus post-PD successful predator discovery checks - Hb & DRN

In [8]:
successfulCheckConstant_prePD = pd.read_csv(f'{root}PPIs_successfulPredatorDiscovery_prePD_CheckOnly_DRN_HB.csv')
checks_postPD = pd.read_csv(f'{root}PPIs_postPD_checkOnly_DRN_HB.csv')

In [12]:
successfulCheckConstant_prePD

,Condition,Regressor,ROI,Seed,Uncorrected_pvalue,Significant,T_stats,df,Avg_peak_sec,Peak_search_window,...,Peak_vals_14,Peak_vals_15,Peak_vals_16,Peak_vals_17,Peak_vals_18,Peak_vals_19,Peak_vals_20,Peak_vals_21,Peak_vals_22,Peak_vals_23
0,vigilance_successfulChecks_outcome,timepressure,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.002,1,3.425,22,3.503,0.06s-4.98s,...,0.342,0.111,-0.232,0.081,0.294,-0.058,0.191,0.496,0.247,-0.102
1,vigilance_successfulChecks_outcome,time,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.079,0,-1.844,22,2.029,0.06s-4.98s,...,0.151,0.050,0.053,-0.034,-0.176,0.262,0.002,-0.157,-0.150,-0.120
2,vigilance_successfulChecks_outcome,reward,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.043,1,-2.145,22,1.538,0.06s-4.98s,...,-0.164,-0.140,-0.210,-0.095,-0.043,0.299,-0.289,-0.049,0.098,-0.044
3,vigilance_successfulChecks_outcome,constant,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.000,1,-4.142,22,4.977,0.06s-4.98s,...,0.397,-0.419,-0.180,-0.175,-0.010,-0.367,-0.196,0.058,-0.065,-0.216
4,vigilance_successfulChecks_outcome,timepressure,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.245,0,1.194,22,5.173,4.98s-9.99s,...,0.364,0.256,-0.002,-0.048,0.314,0.082,0.052,-0.168,0.252,-0.146
5,vigilance_successfulChecks_outcome,time,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.205,0,1.306,22,5.272,4.98s-9.99s,...,0.093,-0.254,0.097,-0.038,-0.214,-0.190,0.219,0.227,-0.133,-0.010
6,vigilance_successfulChecks_outcome,reward,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.282,0,-1.103,22,8.711,4.98s-9.99s,...,-0.064,-0.277,-0.112,-0.094,-0.289,-0.094,0.075,0.270,-0.055,-0.132
7,vigilance_successfulChecks_outcome,constant,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.001,1,-3.896,22,5.075,4.98s-9.99s,...,0.409,-0.426,-0.184,-0.175,-0.010,-0.371,-0.193,0.091,-0.055,-0.212
8,vigilance_successfulChecks_outcome,timepressure,HB,none,0.002,1,3.472,22,4.977,0.06s-4.98s,...,0.365,0.101,-0.253,0.082,0.096,-0.265,0.085,-0.007,0.149,0.151
9,vigilance_successfulChecks_outcome,time,HB,none,0.176,0,1.400,22,0.653,0.06s-4.98s,...,-0.268,-0.063,-0.107,-0.043,0.043,0.123,-0.082,0.110,0.011,0.347


In [18]:
# Hb (whole window)
cols = [x for x in checks_postPD.columns if 'Peak_vals' in x]

ttest_ind(
    a = checks_postPD.loc[
        (checks_postPD.Regressor=='constant') &
        (checks_postPD.ROI=='HB'),cols
    ].values[0],
    b = successfulCheckConstant_prePD.loc[
        (successfulCheckConstant_prePD.Regressor=='constant') &
        (successfulCheckConstant_prePD.ROI=='HB'),cols
    ].values[0],
nan_policy='omit')

TtestResult(statistic=-0.769628771268146, pvalue=0.4456324072458351, df=44.0)

In [20]:
# DRN (t-test early window)
ttest_ind(
    a = checks_postPD.loc[
        (checks_postPD.Regressor=='constant') &
        (checks_postPD.ROI=='DRN_sub_PAG_edit_sub_SC_edit_ero'),cols
    ].values[0],
    b = successfulCheckConstant_prePD.loc[
        (successfulCheckConstant_prePD.Regressor=='constant') &
        (successfulCheckConstant_prePD.ROI=='DRN_sub_PAG_edit_sub_SC_edit_ero'),cols
    ].values[0],
nan_policy='omit')

TtestResult(statistic=1.4576213079287523, pvalue=0.1520458079819331, df=44.0)

### Test pre-PD versus post-PD successful predator discovery checks - Hb & DRN

In [17]:
successfulCheckConstant_prePD = pd.read_csv(f'{root}PPIs_successfulPredatorDiscovery_prePD_CheckOnly_DRN_HB.csv')
checks_postPD = pd.read_csv(f'{root}PPIs_postPD_checkOnly_DRN_HB.csv')

In [18]:
successfulCheckConstant_prePD

,Condition,Regressor,ROI,Seed,Uncorrected_pvalue,Significant,T_stats,df,Avg_peak_sec,Peak_search_window,...,Peak_vals_14,Peak_vals_15,Peak_vals_16,Peak_vals_17,Peak_vals_18,Peak_vals_19,Peak_vals_20,Peak_vals_21,Peak_vals_22,Peak_vals_23
0,vigilance_successfulChecks_outcome,timepressure,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.002,1,3.425,22,3.503,0.06s-4.98s,...,0.342,0.111,-0.232,0.081,0.294,-0.058,0.191,0.496,0.247,-0.102
1,vigilance_successfulChecks_outcome,time,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.079,0,-1.844,22,2.029,0.06s-4.98s,...,0.151,0.050,0.053,-0.034,-0.176,0.262,0.002,-0.157,-0.150,-0.120
2,vigilance_successfulChecks_outcome,reward,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.043,1,-2.145,22,1.538,0.06s-4.98s,...,-0.164,-0.140,-0.210,-0.095,-0.043,0.299,-0.289,-0.049,0.098,-0.044
3,vigilance_successfulChecks_outcome,constant,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.000,1,-4.142,22,4.977,0.06s-4.98s,...,0.397,-0.419,-0.180,-0.175,-0.010,-0.367,-0.196,0.058,-0.065,-0.216
4,vigilance_successfulChecks_outcome,timepressure,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.245,0,1.194,22,5.173,4.98s-9.99s,...,0.364,0.256,-0.002,-0.048,0.314,0.082,0.052,-0.168,0.252,-0.146
5,vigilance_successfulChecks_outcome,time,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.205,0,1.306,22,5.272,4.98s-9.99s,...,0.093,-0.254,0.097,-0.038,-0.214,-0.190,0.219,0.227,-0.133,-0.010
6,vigilance_successfulChecks_outcome,reward,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.282,0,-1.103,22,8.711,4.98s-9.99s,...,-0.064,-0.277,-0.112,-0.094,-0.289,-0.094,0.075,0.270,-0.055,-0.132
7,vigilance_successfulChecks_outcome,constant,DRN_sub_PAG_edit_sub_SC_edit_ero,none,0.001,1,-3.896,22,5.075,4.98s-9.99s,...,0.409,-0.426,-0.184,-0.175,-0.010,-0.371,-0.193,0.091,-0.055,-0.212
8,vigilance_successfulChecks_outcome,timepressure,HB,none,0.002,1,3.472,22,4.977,0.06s-4.98s,...,0.365,0.101,-0.253,0.082,0.096,-0.265,0.085,-0.007,0.149,0.151
9,vigilance_successfulChecks_outcome,time,HB,none,0.176,0,1.400,22,0.653,0.06s-4.98s,...,-0.268,-0.063,-0.107,-0.043,0.043,0.123,-0.082,0.110,0.011,0.347


In [22]:
# Threat regressor in Hb: Different from zero for pre-PD?
cols = [x for x in checks_postPD.columns if 'Peak_vals' in x]

ttest_1samp(a = successfulCheckConstant_prePD.loc[
        (successfulCheckConstant_prePD.Regressor=='constant') &
        (successfulCheckConstant_prePD.ROI=='DRN_sub_PAG_edit_sub_SC_edit_ero'),cols
    ].values[0], popmean=0)

TtestResult(statistic=-4.141910210253197, pvalue=0.0004266303865200051, df=22)

In [27]:
successfulCheckConstant_prePD.loc[
        (successfulCheckConstant_prePD.Regressor=='constant') &
        (successfulCheckConstant_prePD.ROI=='DRN_sub_PAG_edit_sub_SC_edit_ero'),cols
    ].values

array([[-0.40955685, -0.41793872, -0.0229184 , -0.12767133, -0.25914342,
        -0.24158923,  0.16865463, -0.13207408, -0.13315062, -0.41834904,
        -0.29958834, -0.21018157, -0.30918535,  0.39662095, -0.4187343 ,
        -0.17997141, -0.17525501, -0.00989485, -0.36674032, -0.19557636,
         0.05803266, -0.06485144, -0.21551162],
       [-0.40296281, -0.36405611, -0.03571262, -0.12767133, -0.25914342,
        -0.23213903,  0.16865463, -0.13438519, -0.13665935, -0.40680621,
        -0.29958834, -0.08875831, -0.30861124,  0.40860112, -0.42592302,
        -0.18448067, -0.17525501, -0.00989485, -0.37068805, -0.19318301,
         0.09139554, -0.05516304, -0.21237105]])